In [5]:
import xmlrpc.client
import xlrd

In [184]:



class ClassOdoo():
    
    def __init__ (self,login,password):
        self.user = login
        self.passw=password
        self.url = 'https://cafe11.odoo.com'
        self.db = 'cafe11'        
        self.common=xmlrpc.client.ServerProxy('{}/xmlrpc/2/common'.format(self.url))
        self.uid=self.common.authenticate(self.db,self.user,self.passw,{})
        self.models=xmlrpc.client.ServerProxy('{}/xmlrpc/2/object'.format(self.url))
        self.models.execute_kw(self.db,self.uid,self.passw,'res.partner','check_access_rights',['read'],{'raise_exception':False})  
        self.TAXRATE = 1.2
    
    def change_attribute(self,product,attribute,new_value):
        ids = self.models.execute_kw(self.db, self.uid, self.passw,'product.product', 'search',[[['name','=',product]]])
        self.models.execute_kw(self.db,self.uid,self.passw,'product.template','write',[ids,{attribute: new_value}])
    
    def create_session(self):
        ids = self.models.execute_kw(self.db, self.uid, self.passw,'pos.session', 'create',[{'company_id': [1, 'cafe'],
    'config_id': 1}])
#time.sleep(20)

    def close_session(self):
        ids=self.models.execute_kw(self.db, self.uid, self.passw,'pos.session', 'search',[[['state','=','opened']]])
        self.models.execute_kw(self.db, self.uid, self.passw,'pos.session', 'write',[ids,{'state':'closed'}])
   
    def pull_open_session(self):
        ids = self.models.execute_kw(self.db, self.uid, self.passw,'pos.session', 'search',[[['state','=','opened']]])
        record = self.models.execute_kw(self.db, self.uid, self.passw,'pos.session', 'read', [ids])
    
        return record
    
    def pull_all_session(self):
        ids = self.models.execute_kw(self.db, self.uid, self.passw,'pos.session', 'search',[[]])
        record = self.models.execute_kw(self.db, self.uid, self.passw,'pos.session', 'read', [ids])
    
        return record
    
    def getIDPrice(self,productsList:list) -> dict:
        
        ids = self.models.execute_kw(self.db, self.uid, self.passw,'product.product', 'search',[[['available_in_pos','=',True],['name','=',productsList]]])
        
        record = self.models.execute_kw(self.db, self.uid, self.passw,'product.product', 'read', [ids],{'fields': ['list_price','id','name']})
        
        #print(record)
        
        return record
    def pull_all_orders(self):
        ids = self.models.execute_kw(self.db, self.uid, self.passw,'pos.order', 'search',[[]])
        record = self.models.execute_kw(self.db, self.uid, self.passw,'pos.order', 'read', [ids])
    
        return record

    
    def push_order(self,productDict):
        
        
             
        products = self.getIDPrice(list(productDict.keys())) 
        
        ids = self.models.execute_kw(self.db, self.uid, self.passw,'pos.session', 'search',[[['state','=','opened']]])
        id_order = self.models.execute_kw(self.db, self.uid, self.passw,'pos.order', 'create',[{'session_id': ids[0],'amount_tax':0,'amount_total':0,'amount_paid':0,'amount_return':0}])
       
        
            
        for product in products:
            product['qty'] = productDict[product['name']]
            product['price_subtotal'] = product['list_price'] * product['qty']
            product['price_subtotal_incl'] = product['price_subtotal'] * self.TAXRATE
            product['order_id'] = id_order
            product['price_unit']=product['list_price']
            del product['name']
            del product['list_price']
            product['product_id'] = product['id']
            del product['id']
        
        self.models.execute_kw(self.db, self.uid, self.passw,'pos.order.line', 'create',[products]) 
                                                                                               
    def get_fields(self,model):
        return self.models.execute_kw(self.db, self.uid, self.passw, model, 'fields_get',[], {'attributes': ['string', 'help', 'type']})
        
     
    
    def pull_products_for_order(self,order_id):
        ids=self.models.execute_kw(self.db, self.uid, self.passw,'pos.order.line', 'search', [[['order_id','=',order_id]]])
        record = self.models.execute_kw(self.db, self.uid, self.passw,'pos.order.line', 'read', [ids])
        return record
        
   
    def get_menu(self):
        ids = self.models.execute_kw(self.db, self.uid, self.passw,'product.product', 'search',[[['available_in_pos','=',True]]])
        record = self.models.execute_kw(self.db, self.uid, self.passw,'product.product', 'read', [ids],{'fields': ['display_name','list_price','categ_id','taxes_id']})
        return record
    
    def create_products(self,fields,values):
        dict_=dict(zip(fields,values))
        ids = self.models.execute_kw(self.db, self.uid, self.passw,'product.product', 'create',[dict_])
        
        
    def delete(self,product):
        ids = self.models.execute_kw(self.db, self.uid, self.passw,'product.product', 'search',[[['name','=',product]]])
        self.models.execute_kw(self.db, self.uid, self.passw, 'product.product', 'unlink', [ids])
        
    def push_payment(self,order_id,amount_paid):
        record=self.pull_products_for_order(order_id)
        amount_total=0
        amount_tax=0
        
        lenght=len([i for i in record if i])
        for i in range(lenght):
            amount_total+=record[i]['price_subtotal_incl']
            amount_tax+=amount_total-record[i]['price_subtotal']
            
        amount_returned=amount_paid-amount_total
        if amount_returned>0:
            self.models.execute_kw(self.db, self.uid, self.passw,'pos.order', 'write',[order_id,{'amount_tax':amount_tax,'amount_total':amount_total,'amount_paid':amount_paid,'amount_return':amount_returned,'state':'paid'}])
            return
        print("Payment declined due to insufficient funds")

In [185]:
odoo=ClassOdoo("a.marzetta@yahoo.com","ciao")

product_array=['Hamburger','Pasta al Pesto']
quanity_array=[6,4]



In [186]:
odoo.push_payment(2,100)
#odoo.push_payment(1,100)

In [187]:
#products = {'Hamburger':10,'Pasta al Pesto':12}

In [188]:
#odoo.getIDPrice(product_array)

In [189]:
#odoo.push_order(products)